In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1277,tt2058107,18000000,22309223,The Railway Man,Jeremy Irvine|Colin Firth|Stellan SkarsgÃ¥rd|M...,Jonathan Teplitzky,Revenge is never a straight line,"A victim from World War II's ""Death Railway"" s...",116,Drama|History,Archer Street Productions|Lionsgate|Pictures i...,12/26/2013,6.6,2013
1111,tt0327056,25000000,156822020,Mystic River,Sean Penn|Tim Robbins|Kevin Bacon|Marcia Gay H...,Clint Eastwood,"We bury our sins, we wash them clean.","A story about friendship and loyalty, guilt an...",138,Thriller|Crime|Drama|Mystery,Village Roadshow Pictures|Malpaso Productions|...,10/7/2003,7.4,2003
343,tt1220198,10000000,42333295,The Fourth Kind,Milla Jovovich|Will Patton|Hakeem Kae-Kazim|Co...,Olatunde Osunsanmi,There are four kinds of alien encounters. The ...,"Since the 1960s, a disproportionate number of ...",98,Science Fiction|Thriller,Universal Pictures|Saga Film|Focus Films|Gold ...,11/6/2009,5.6,2009
431,tt1231583,65000000,211780824,Due Date,Zach Galifianakis|Robert Downey Jr.|Michelle M...,Todd Phillips,Leave Your Comfort Zone,Peter Highman must scramble across the US in f...,95,Comedy|Drama,Legendary Pictures|Green Hat Films|Warner Bros.,11/4/2010,6.2,2010
866,tt0258463,60000000,214034224,The Bourne Identity,Matt Damon|Franka Potente|Chris Cooper|Clive O...,Doug Liman,He was the perfect weapon until he became the ...,Wounded to the brink of death and suffering fr...,119,Action|Drama|Mystery|Thriller,"Universal Pictures|Kennedy/Marshall Company, T...",6/14/2002,7.1,2002


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# the time given in the dataset is in string format.
# So we need to change this in datetime format
data.release_date = pd.to_datetime(data.release_date)

# create custom profit column
data['profit'] = data.revenue - data.budget

# add seconds columns that contain splitted data of original columns
# it allows applying .explode function
listed_columns = ['cast', 'director', 'genres', 'production_companies']
for column in listed_columns:
    data[column + '_list'] = data[column].str.split('|')

# 1. У какого фильма из списка самый большой бюджет?

In [5]:
movie = data.sort_values('budget', ascending=False).iloc[0]

movie_title = movie['original_title']
movie_id = movie['imdb_id']
answers['1'] = '{} ({})'.format(movie_title, movie_id)
print(answers['1'])

Pirates of the Caribbean: On Stranger Tides (tt1298650)


ВАРИАНТ 2

In [6]:
movie = data.loc[data.budget.idxmax()]

movie_title = movie.original_title
movie_id = movie.imdb_id
answers['1'] = '{} ({})'.format(movie_title, movie_id)
print(answers['1'])

Pirates of the Caribbean: On Stranger Tides (tt1298650)


# 2. Какой из фильмов самый длительный (в минутах)?

In [7]:
movie = data.loc[data.runtime.idxmax()]

movie_title = movie.original_title
movie_id = movie.imdb_id
answers['2'] = '{} ({})'.format(movie_title, movie_id)
print(answers['2'])

Gods and Generals (tt0279111)


# 3. Какой из фильмов самый короткий (в минутах)?





In [8]:
movie = data.loc[data.runtime.idxmin()]

movie_title = movie.original_title
movie_id = movie.imdb_id
answers['3'] = '{} ({})'.format(movie_title, movie_id)
print(answers['3'])

Winnie the Pooh (tt1449283)


# 4. Какова средняя длительность фильмов?


In [9]:
movie_mean = data.runtime.mean()

answers['4'] = '{:.0f}'.format(movie_mean)
print(answers['4'])

110


# 5. Каково медианное значение длительности фильмов? 

In [10]:
movie_median = data.runtime.median()

answers['5'] = int(movie_median)
print(answers['5'])

107


# 6. Какой самый прибыльный фильм?

In [11]:
movie = data.loc[data.profit.idxmax()]

movie_title = movie.original_title
movie_id = movie.imdb_id
answers['6'] = '{} ({})'.format(movie_title, movie_id)
print(answers['6'])

Avatar (tt0499549)


# 7. Какой фильм самый убыточный? 

In [12]:
movie = data.loc[data.profit.idxmin()]

movie_title = movie.original_title
movie_id = movie.imdb_id
answers['7'] = '{} ({})'.format(movie_title, movie_id)
print(answers['7'])

The Lone Ranger (tt1210819)


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [13]:
positive_profit_movies = data[data.profit > 0]

answers['8'] = len(positive_profit_movies)
print(answers['8'])

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [14]:
movies = data[data.release_year == 2008]
movie = movies.loc[movies.profit.idxmax()]

movie_title = movie.original_title
movie_id = movie.imdb_id
answers['9'] = '{} ({})'.format(movie_title, movie_id)
print(answers['9'])

The Dark Knight (tt0468569)


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [15]:
movies = data[(data.release_year >= 2012) & (data.release_year <= 2014)]
movie = movies.loc[movies.profit.idxmin()]

movie_title = movie.original_title
movie_id = movie.imdb_id
answers['10'] = '{} ({})'.format(movie_title, movie_id)
print(answers['10'])

The Lone Ranger (tt1210819)


# 11. Какого жанра фильмов больше всего?

In [16]:
genres = data.genres_list.apply(pd.Series).stack().unique()

counter = Counter()
for genre in genres:
    counter[genre] = len(data[data.genres.str.contains(genre)])
            
result = counter.most_common(1)[0][0]

answers['11'] = result
print(answers['11'])

Drama


ВАРИАНТ 2

In [17]:
result = data.explode('genres_list') \
    .groupby('genres_list') \
    .imdb_id \
    .count()

answers['11'] = result.idxmax()
print(answers['11'])

Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [18]:
result = data.explode('genres_list') \
    .groupby('genres_list') \
    .profit \
    .count()

answers['12'] = result.idxmax()
print(answers['12'])

Drama


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [19]:
result = data.explode('director_list') \
    .groupby('director_list') \
    .profit \
    .sum()

answers['13'] = result.idxmax()
print(answers['13'])

Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [20]:
result = data[data.genres.str.contains('Action')] \
    .explode('director_list') \
    .groupby('director_list') \
    .imdb_id \
    .count()

answers['14'] = result.idxmax()
print(answers['14'])

Robert Rodriguez


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [21]:
result = data[data.release_date.dt.year == 2012] \
    .explode('cast_list') \
    .groupby('cast_list') \
    .profit \
    .sum()

answers['15'] = result.idxmax()
print(answers['15'])

Chris Hemsworth


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [22]:
result = data[data.budget > data.budget.mean()] \
    .explode('cast_list') \
    .groupby('cast_list') \
    .imdb_id \
    .count()

answers['16'] = result.idxmax()
print(answers['16'])

Matt Damon


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [23]:
result = data[data.cast.str.contains('Nicolas Cage')] \
    .explode('genres_list') \
    .groupby('genres_list') \
    .imdb_id \
    .count()

answers['17'] = result.idxmax()
print(answers['17'])

Action


# 18. Самый убыточный фильм от Paramount Pictures

In [24]:
company = 'Paramount Pictures'

result = data[data.production_companies.str.contains(company)]
result = result[result.profit == result.profit.min()]

movie = result.iloc[0]
movie_title = movie.original_title
movie_id = movie.imdb_id

answers['18'] = '{} ({})'.format(movie_title, movie_id)
print(answers['18'])

K-19: The Widowmaker (tt0267626)


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [25]:
result = data.groupby(data.release_date.dt.year).profit.sum()

answers['19'] = result.idxmax()
print(answers['19'])

2015


# 20. Какой самый прибыльный год для студии Warner Bros?

In [26]:
company = 'Warner Bros'
result = data[data.production_companies.str.contains(company)]
result = result.groupby(data.release_date.dt.year).profit.sum()

answers['20'] = result.idxmax()
print(answers['20'])

2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [27]:
month_group = data.groupby(data.release_date.dt.month_name())
month = month_group.imdb_id.count().idxmax()

answers['21'] = month
print(answers['21'])

September


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [28]:
month = data.release_date.dt.month
summer_filter = (month >=6) & (month <=8)

movie_count = len(data[summer_filter].imdb_id)

answers['22'] = movie_count
print(answers['22'])

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [29]:
month = data.release_date.dt.month
result = data[(month == 12) | (month <= 2)] \
    .explode('director_list') \
    .groupby('director_list') \
    .imdb_id \
    .count()

answers['23'] = result.idxmax()
print(answers['23'])

Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [30]:
data['original_title_len'] = data.original_title.str.len()
result = data \
    .explode('production_companies_list') \
    .groupby('production_companies_list') \
    .original_title_len \
    .mean()

answers['24'] = result.idxmax()
print(answers['24'])

Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [31]:
count_words = lambda x: len(re.findall('\w+', x))
data['overview_words_count'] = data.overview.apply(count_words)

result = data \
    .explode('production_companies_list') \
    .groupby('production_companies_list') \
    .overview_words_count \
    .mean()

answers['25'] = result.idxmax()
print(answers['25'])

Midnight Picture Show


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [32]:
result = data.quantile(0.99)['vote_average']
result = data[data.vote_average >= result] \
    [['original_title', 'vote_average']] \
    .sort_values(by='vote_average', ascending=False)

answers['26'] = result.original_title.to_list()
result

,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
125,The Imitation Game,8.0
370,Inception,7.9
1191,12 Years a Slave,7.9
1183,The Wolf of Wall Street,7.9
1081,The Lord of the Rings: The Return of the King,7.9
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?

In [37]:
df = data[['imdb_id', 'cast_list']].explode('cast_list')
actors = [(x, y) for _, d in df.groupby('imdb_id') \
          for x, y in combinations(d['cast_list'], 2)]

result = pd.value_counts(actors)

display(result.sort_values(ascending=False).head(5))

answers['27'] = result.idxmax()
print(answers['27'])

(Daniel Radcliffe, Rupert Grint)       8
(Daniel Radcliffe, Emma Watson)        8
(Rupert Grint, Emma Watson)            7
(Ben Stiller, Owen Wilson)             6
(Johnny Depp, Helena Bonham Carter)    6
dtype: int64

('Daniel Radcliffe', 'Rupert Grint')


# Submission

In [34]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['The Dark Knight',
  'Inside Out',
  'Room',
  'Interstellar',
  'The Imitation Game',
  'Inception',
  '12 Years a Slave',
  'The Wolf of Wall Street',
  'The Lord of the Rings: The Return of the King',
  'The Pianist',
  'Memento',
  'Guardians of the Galaxy',
  'The Grand Budapest Hotel',
  'Gone Girl',
  'Mr. Nobody',
  'There Will

In [35]:
# и убедиться что ни чего не пропустил)
len(answers)

27